In [1]:
!pip3 install ktrain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.3/25.3 MB 40.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 51.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 263.7/263.7 kB 21.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.8/468.8 kB 34.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
  Created wheel for ktrain: filename=ktrain-0.33.0-py3-none-any.whl size=25313832 sha256=fb8b627fa8e8c6b66391e6e77cc8202744231fa58002edefeff9ebcb9c3b4a7e
  Stored in directory: /root/.cache/pip/wheels/cb/c2/1a/db7e1066fb4a94be

In [2]:
!pip install nltk

In [3]:
import pandas as pd
import numpy as np

import ktrain
from ktrain import text
import nltk

In [4]:
from torch.utils.data import TensorDataset,RandomSampler,SequentialSampler
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
import spacy
import re,string,unicodedata

In [5]:
path = '/kaggle/input/d/sriyar26/sexism-dataset/train_all_tasks.csv'

data_train = pd.read_csv(path)

In [6]:
path1 = '/kaggle/input/d/sriyar26/sexism-dataset/datasets/dev_task_a_entries.csv'

data_test = pd.read_csv(path1)

In [7]:
data_train

,rewire_id,text,label_sexist,label_category,label_vector
0,sexism2022_english-7358,"Damn, this writing was pretty chaotic",not sexist,none,none
1,sexism2022_english-2367,"Yeah, and apparently a bunch of misogynistic v...",not sexist,none,none
2,sexism2022_english-3073,How the FUCK is this woman still an MP!!!???,not sexist,none,none
3,sexism2022_english-14895,Understand. Know you're right. At same time I ...,not sexist,none,none
4,sexism2022_english-4118,Surprized they didn't stop and rape some women,not sexist,none,none
...,...,...,...,...,...
13995,sexism2022_english-14185,complexes like the 'nice chicks' that go after...,sexist,2. derogation,2.1 descriptive attacks
13996,sexism2022_english-18629,"""GRAPHIC Germany - Muslim ""refugee"" stabbing h...",not sexist,none,none
13997,sexism2022_english-3320,Lol I imagine there would be simps that are li...,not sexist,none,none
13998,sexism2022_english-13309,"It's not, the girls I go on dates with don't k...",not sexist,none,none


In [8]:
data_test.head(2)

,rewire_id,text
0,sexism2022_english-11999,I do not try to hard. I do what guys who get w...
1,sexism2022_english-9895,That sucks. We elected a Navy Seal in Missouri...


In [9]:
data_train['label_sexist'].value_counts()

not sexist    10602
sexist         3398
Name: label_sexist, dtype: int64

In [10]:
classes = ['sexist', 'not sexist']


In [11]:
import re
def clean_text(text):

    text = re.sub(r"@[A-Za-z0-9]+", ' ', text)
    text = re.sub(r"https?://[A-Za-z0-9./]+", ' ', text)
    text = re.sub(r"[^a-zA-z.!?'0-9]", ' ', text)
    text = re.sub('\t', ' ',  text)
    text = re.sub(r" +", ' ', text)
    text=re.sub(r'[^\w\s]','',text)
    text=text.replace("[URL]","")
    text=text.replace("[USER]","")
    text=re.sub(r"[â€™ºðŸ‡˜Žµ±¤£‘Œ”œ]","",text)
    return text

In [12]:
data_train['text']= data_train['text'].apply(clean_text)

In [13]:
#Removing the square brackets
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)

#Removing the noisy text
def denoise_text(text):
    
    text = remove_between_square_brackets(text)
    return text

In [14]:
data_train['text']=data_train['text'].apply(denoise_text)

In [15]:
from sklearn.model_selection import train_test_split
X=data_train.text.tolist()
y=data_train.label_sexist.tolist()

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [17]:
encoding = {
    'sexist': 0,
    'not sexist': 1
}

# Integer values for each class
y_train = [encoding[x] for x in y_train]
y_test = [encoding[x] for x in y_test]

In [18]:
import tensorflow as tf
from tensorflow.keras import activations
def focal_loss(gamma=2., alpha=4., from_logits=False):

    gamma = float(gamma)
    alpha = float(alpha)

    def focal_loss_fixed(y_true, y_pred):
        """Focal loss for multi-classification
        FL(p_t)=-alpha(1-p_t)^{gamma}ln(p_t)
        Notice: y_pred is probability after softmax if from_logits is False.
        gradient is d(Fl)/d(p_t) not d(Fl)/d(x) as described in paper
        d(Fl)/d(p_t) * [p_t(1-p_t)] = d(Fl)/d(x)
        Focal Loss for Dense Object Detection
        https://arxiv.org/abs/1708.02002

        Arguments:
            y_true {tensor} -- ground truth labels, shape of [batch_size, num_cls]
            y_pred {tensor} -- model's output, shape of [batch_size, num_cls]

        Keyword Arguments:
            gamma {float} -- (default: {2.0})
            alpha {float} -- (default: {4.0})

        Returns:
            [tensor] -- loss.
        """
        epsilon = 1.e-9
        y_true = tf.cast(y_true, dtype=tf.float32)
        y_pred = tf.cast(y_pred, dtype=tf.float32)
        if from_logits:
            y_pred = activations.softmax(y_pred)

        model_out = tf.add(y_pred, epsilon)
        ce = tf.multiply(y_true, -tf.math.log(model_out))
        weight = tf.multiply(y_true, tf.pow(tf.subtract(1., model_out), gamma))
        fl = tf.multiply(alpha, tf.multiply(weight, ce))
        reduced_fl = tf.reduce_max(fl, axis=1)
        return tf.reduce_mean(reduced_fl)
    return focal_loss_fixed

In [19]:
from ktrain import text
BATCH_SIZE = 6
MODEL_NAME = 'microsoft/deberta-v3-base'

t = text.Transformer(MODEL_NAME, maxlen=500, class_names=[0, 1])
trn = t.preprocess_train(X_train, y_train)
val = t.preprocess_test(X_test, y_test)

model = t.get_classifier()

Downloading:   0%|          | 0.00/579 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/702M [00:00<?, ?B/s]

preprocessing train...
language: en
train sequence lengths:
	mean : 23
	95percentile : 43
	99percentile : 48


Downloading:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.35M [00:00<?, ?B/s]

/opt/conda/lib/python3.7/site-packages/transformers/convert_slow_tokenizer.py:435: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  "The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option"


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 24
	95percentile : 44
	99percentile : 48


In [20]:
model.compile(loss=focal_loss(alpha=1, from_logits=True),optimizer='adam',metrics=['accuracy'])

In [21]:
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=BATCH_SIZE)

In [22]:
learner.fit_onecycle(4e-5, 6) #use hardware acceleartor while running this cell



begin training using onecycle policy with max lr of 2e-05...
Epoch 1/6
1867/1867 [==============================] - 2669s 1s/step - loss: 0.1147 - accuracy: 0.8033 - val_loss: 0.0952 - val_accuracy: 0.8468
Epoch 2/6
1867/1867 [==============================] - 2601s 1s/step - loss: 0.0788 - accuracy: 0.8785 - val_loss: 0.0925 - val_accuracy: 0.8550
Epoch 3/6
1867/1867 [==============================] - 2598s 1s/step - loss: 0.0633 - accuracy: 0.9038 - val_loss: 0.1128 - val_accuracy: 0.8461
Epoch 4/6
1867/1867 [==============================] - 2603s 1s/step - loss: 0.0412 - accuracy: 0.9430 - val_loss: 0.1179 - val_accuracy: 0.8671
Epoch 5/6
1867/1867 [==============================] - 2603s 1s/step - loss: 0.0164 - accuracy: 0.9787 - val_loss: 0.1807 - val_accuracy: 0.8646
Epoch 6/6
1867/1867 [==============================] - 2602s 1s/step - loss: 0.0047 - accuracy: 0.9946 - val_loss: 0.2690 - val_accuracy: 0.8639


In [23]:
learner.validate(class_names=classes)

              precision    recall  f1-score   support

      sexist       0.75      0.69      0.72       704
  not sexist       0.90      0.92      0.91      2096

    accuracy                           0.86      2800
   macro avg       0.82      0.81      0.81      2800
weighted avg       0.86      0.86      0.86      2800



array([[ 487,  217],
       [ 164, 1932]])

In [24]:
x_valid=data_test["text"]

In [25]:
from unicodedata import normalize


In [26]:
df = pd.DataFrame()

In [27]:
predictor = ktrain.get_predictor(learner.model, preproc=t)
predictor.get_classes()

[0, 1]

In [28]:
preds=[]
for entry in x_valid:
  normalize('NFKD', entry).encode('ascii','ignore')
  preds.append(predictor.predict(entry))

In [29]:
preds = ['sexist' if element == 0 else 'not sexist' for element in preds]

In [30]:
submission=pd.DataFrame({
    "rewire_id":data_test['rewire_id'],
    "label_pred":preds
})

In [31]:
submission.to_csv('DevAdebertav3focal.csv',index=False)

In [32]:
#prediction = predictor.predict(message)

#print('Emotion predicted: {} '.format(prediction))